Problémy:
- za USA data jen do podzimu 2021
- různé číslování týdnů USA/ČR
- časový ořez – Mariah Carey byla u nás prý nasazená výrazně později než v USA, ale to my nevíme, páč hrála už před rokem 2006 (v závěrečném srovnání snad vyřešeno)

In [187]:
import os
import pandas as pd

In [188]:
cz = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
b100 = pd.read_csv(os.path.join("data_b100","b100.csv"))

## Oříznem B100

In [189]:
b100 = b100[b100['rok'] >= 2006]

## Sjednotíme si týdny

In [190]:
cz = cz.sort_values(by = ['rok','tyden','pozice'], ascending=True)

In [191]:
cz['interpret_hlavni'] = cz['interpret_hlavni'].apply(lambda x: x.lower())

In [192]:
cz['wk'] = cz['week'].rank(method='dense').astype(int)

In [193]:
b100['week'] = b100.apply(lambda row: f"""{row['rok']}-{row['tyden']:02}""", axis = 1)

In [194]:
b100['interpret_hlavni'] = b100['interpret_hlavni'].apply(lambda x: x.lower())

In [195]:
cz = cz.sort_values(by="week")
b100 = b100.sort_values(by="week")

### Kontrola

In [196]:
[x for x in b100['week'].drop_duplicates().to_list() if x not in cz['week'].drop_duplicates().to_list()]

['2006-51',
 '2007-52',
 '2009-51',
 '2010-50',
 '2011-26',
 '2011-51',
 '2011-53',
 '2012-52',
 '2013-52',
 '2014-33',
 '2014-51',
 '2015-52',
 '2016-51',
 '2016-53',
 '2017-51',
 '2018-08',
 '2018-51',
 '2019-30',
 '2020-01']

VELKÝ ŠPATNÝ: PROTOŽE MÁME PRÁZDNINOVÉ A NĚKDY I VÁNOČNÍ PAUZY, POROVNÁNÍ MŮŽE BĚŽNĚ LÍTAT O TÝDEN.

Něco vypovídajícího možná zjistíme, ale nutno na toto explicitně upozornit v textu.

## Porovnání

In [197]:
americke_jednicky = b100[b100['rank'] == 1]['interpret_hlavni'].drop_duplicates().to_list()

In [198]:
americke_jednicky_v_cesku = pd.Series(cz[cz['interpret_hlavni'].isin(americke_jednicky)].groupby('interpret_hlavni')['pozice'].min().reindex(americke_jednicky).sort_values())

### Nejhůře umístěné americké jedničky u nás

In [199]:
americke_jednicky_v_cesku.tail(50)

interpret_hlavni
billie eilish              7.0
lorde                      7.0
usher                      9.0
halsey                     9.0
olivia rodrigo             9.0
robin thicke              10.0
lizzo                     11.0
mariah carey              11.0
john legend               12.0
kendrick lamar            12.0
nelly                     15.0
dj khaled                 16.0
chris brown               19.0
iggy azalea               19.0
kanye west                19.0
jay sean                  20.0
jawsh 685                 21.0
doja cat                  23.0
daniel powter             25.0
drake                     26.0
t-pain                    27.0
ludacris                  32.0
d4l                       41.0
cardi b                   44.0
b.o.b                     67.0
rae sremmurd              98.0
chamillionaire             NaN
taylor hicks               NaN
mims                       NaN
plain white t's            NaN
sean kingston              NaN
soulja boy tell'em    

In [200]:
cz[cz['interpret_hlavni'].str.contains("P!Nk",na=False)]['interpret_hlavni'].drop_duplicates().to_list()

[]

### Zpoždění (?) za USA

In [201]:
b100[b100['rok'] < 2021].groupby('rok')['week'].nunique().mean()

52.13333333333333

In [202]:
def rozdil_tydnu(t1, t2):
    
    try:
    
        rok1 = int(t1[:4])
        tyden1 = int(t1[5:])
        rok2 = int(t2[:4])
        tyden2 = int(t2[5:])

        letorozdil = rok2-rok1
        letorozdil = letorozdil*52

        tydnorozdil = tyden2-tyden1

        return(letorozdil + tydnorozdil)
    
    except:
        
        return(None)

In [221]:
def porovnani(ranking):
    dfp = {}
    for j in b100[b100['rank'] <= ranking]['interpret_hlavni'].drop_duplicates().to_list():
        dfp[j] = [b100[(b100['interpret_hlavni'] == j) & (b100['rank'] <= ranking)]['week'].min(), cz[(cz['interpret_hlavni'] == j) & (cz['pozice'] <= ranking)]['week'].min()]
    dfp = pd.DataFrame.from_dict(dfp).transpose().rename(columns={0: "USA", 1: "ČR"})
    dfp['zpozdeni'] = dfp.apply(lambda row: rozdil_tydnu(row['USA'], row['ČR']), axis = 1)
    dfp = dfp.dropna()
    return dfp

In [222]:
jednicky = porovnani(1)

In [223]:
jednicky

,USA,ČR,zpozdeni
james blunt,2006-10,2006-06,-4.0
rihanna,2006-19,2010-08,197.0
shakira,2006-24,2006-25,1.0
nelly furtado,2006-27,2007-12,37.0
fergie,2006-33,2007-42,61.0
justin timberlake,2006-36,2016-30,514.0
timbaland,2007-16,2010-24,164.0
flo rida,2008-01,2012-29,236.0
coldplay,2008-26,2008-44,18.0
katy perry,2008-27,2008-43,16.0


In [236]:
jednicky[jednicky['USA'] > '2010-01']['zpozdeni'].median()

18.5

In [225]:
desitky = porovnani(10)

In [226]:
desitky

,USA,ČR,zpozdeni
black eyed peas,2006-01,2006-08,7.0
beyoncé,2006-01,2006-15,14.0
fall out boy,2006-02,2018-13,635.0
james blunt,2006-05,2006-04,-1.0
ne-yo,2006-06,2008-36,134.0
...,...,...,...
lil nas x,2019-39,2022-04,121.0
tones and i,2019-51,2019-44,-7.0
saint jhn,2020-24,2020-24,0.0
24kgoldn,2020-37,2021-11,26.0


In [237]:
desitky[desitky['USA'] > '2010-01']['zpozdeni'].median()

7.0

In [228]:
stovky = porovnani(100)

In [229]:
stovky

,USA,ČR,zpozdeni
mariah carey,2006-01,2008-16,119.0
t-pain,2006-01,2011-47,306.0
daddy yankee,2006-01,2006-01,0.0
chris brown,2006-01,2006-13,12.0
busta rhymes,2006-01,2008-21,124.0
...,...,...,...
elton john,2021-35,2018-12,-179.0
ckay,2021-40,2022-28,40.0
skylar grey,2021-42,2013-03,-455.0
oliver tree,2021-43,2022-05,14.0


In [238]:
stovky[stovky['USA'] > '2010-01']['zpozdeni'].median()

5.5